Fundamentos de ciência de dados - Luiz Davi e Thiago Cavalcanti

In [1]:
from nba_api_jogadores_utils import obter_logs_dos_jogos_por_jogador
from constantes_jogadores import JOGADOR_1, JOGADOR_2, JOGADOR_3, JOGADOR_1_ID, JOGADOR_2_ID, JOGADOR_3_ID, TEMPORADA_ATUAL, RESULTADOS_PATH, TIME_ID
import pandas as pd
from nba_api_utils import salvar_dataset_csv

# RF2 -  Para cada jogador do seu time apresente os dados conforme Tabela 2. 
[OBS.: O sistema deve fornecer automaticamente os dados de todos os jogos ocorridos durante a temporada atual em tempo real].

In [2]:
def retornar_estatisticas_do_jogador(logs, nome_jogador):
    logs['PLAYER'] = nome_jogador
    logs['HOME/AWAY'] = logs['MATCHUP'].apply(lambda x: 'Casa' if 'vs.' in x else 'Fora')
    logs['SCOREBOARD'] = None

    logs['WL'] = logs['WL'].map({'W': 'Vitória', 'L': 'Derrota'})
    logs['MATCHUP'] = logs['MATCHUP'].apply(lambda x: x.split(' ')[-1])
    
    dataset = logs[['PLAYER', 'GAME_DATE', 'MATCHUP', 'WL', 'HOME/AWAY', 'PTS', 'REB', 'AST', 'SCOREBOARD', 'Game_ID', 'FG3A', 'FG3M', 'MIN']]
    dataset.columns = ['Jogador', 'Data do jogo', 'Adversário', 'Vitória ou derrota', 'Casa ou fora', 'Pontos', 'Rebotes', 'Assistências', 'Placar', 'Id do jogo', 'Tentativas de cesta de 3 pontos', 'Cestas de 3 pontos marcadas', 'Tempo de permanência']
    return dataset

In [3]:
from nba_api.stats.endpoints import boxscoretraditionalv2

def retornar_placar_da_partida(eh_visitante, id_do_jogo):
    jogo_boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=id_do_jogo)
    dados_boxscore = jogo_boxscore.get_data_frames()[0]

    if eh_visitante:
        placar_adversario = dados_boxscore[dados_boxscore['TEAM_ID'] == TIME_ID]['PTS'].sum()
        placar_time = dados_boxscore[dados_boxscore['TEAM_ID'] != TIME_ID]['PTS'].sum()
    else:
        placar_time = dados_boxscore[dados_boxscore['TEAM_ID'] == TIME_ID]['PTS'].sum()
        placar_adversario = dados_boxscore[dados_boxscore['TEAM_ID'] != TIME_ID]['PTS'].sum()

    return f'{int(placar_time)} - {int(placar_adversario)}'

In [29]:
import time
def adicionar_placares_no_dataset(estatisticas):
    for index, row in estatisticas.iterrows():
        eh_visitante = row['Casa ou fora']
        id_do_jogo = row['Id do jogo']
        
        placar = retornar_placar_da_partida(eh_visitante, id_do_jogo)
        
        time.sleep(1) # Pode ser removido, caso não esteja acontecendo timeout na requisição para a API
        
        estatisticas.at[index, 'Placar'] = placar

## Obtendo estatísticas de cada jogador

In [6]:
logs_jogador_1 = obter_logs_dos_jogos_por_jogador(JOGADOR_1_ID, TEMPORADA_ATUAL)
estatisticas_jogador_1 = retornar_estatisticas_do_jogador(logs_jogador_1, JOGADOR_1)

logs_jogador_2 = obter_logs_dos_jogos_por_jogador(JOGADOR_2_ID, TEMPORADA_ATUAL)
estatisticas_jogador_2 = retornar_estatisticas_do_jogador(logs_jogador_2, JOGADOR_2)

logs_jogador_3 = obter_logs_dos_jogos_por_jogador(JOGADOR_3_ID, TEMPORADA_ATUAL)
estatisticas_jogador_3 = retornar_estatisticas_do_jogador(logs_jogador_3, JOGADOR_3)

In [30]:
adicionar_placares_no_dataset(estatisticas_jogador_1)
estatisticas_jogador_1 = estatisticas_jogador_1.drop(columns=['Id do jogo'])

estatisticas_jogador_1

,Jogador,Data do jogo,Adversário,Vitória ou derrota,Casa ou fora,Pontos,Rebotes,Assistências,Placar,Tentativas de cesta de 3 pontos,Cestas de 3 pontos marcadas,Tempo de permanência
0,Nikola Jokić,"JAN 29, 2025",NYK,Derrota,Fora,17,6,6,122 - 112,7,3,33
1,Nikola Jokić,"JAN 27, 2025",CHI,Derrota,Fora,33,12,14,129 - 121,4,1,40
2,Nikola Jokić,"JAN 25, 2025",MIN,Derrota,Fora,20,3,11,133 - 104,4,2,32
3,Nikola Jokić,"JAN 23, 2025",SAC,Vitória,Casa,35,22,17,123 - 132,3,2,37
4,Nikola Jokić,"JAN 21, 2025",PHI,Vitória,Casa,27,13,10,109 - 144,3,2,30
5,Nikola Jokić,"JAN 19, 2025",ORL,Vitória,Fora,20,14,10,100 - 113,0,0,30
6,Nikola Jokić,"JAN 17, 2025",MIA,Vitória,Fora,24,12,10,113 - 133,2,1,31
7,Nikola Jokić,"JAN 14, 2025",DAL,Vitória,Fora,10,14,10,99 - 118,2,1,29
8,Nikola Jokić,"JAN 12, 2025",DAL,Vitória,Fora,19,18,9,101 - 112,2,0,39
9,Nikola Jokić,"JAN 10, 2025",BKN,Vitória,Casa,35,12,15,105 - 124,1,1,38


In [17]:
adicionar_placares_no_dataset(estatisticas_jogador_2)
estatisticas_jogador_2 = estatisticas_jogador_2.drop(columns=['Id do jogo'])

estatisticas_jogador_2

,Jogador,Data do jogo,Adversário,Vitória ou derrota,Casa ou fora,Pontos,Rebotes,Assistências,Placar,Tentativas de cesta de 3 pontos,Cestas de 3 pontos marcadas,Tempo de permanência
0,Michael Porter Jr.,"JAN 29, 2025",NYK,Derrota,Fora,9,3,2,122 - 112,5,1,26
1,Michael Porter Jr.,"JAN 27, 2025",CHI,Derrota,Fora,18,7,0,129 - 121,8,2,32
2,Michael Porter Jr.,"JAN 25, 2025",MIN,Derrota,Fora,11,0,2,133 - 104,6,3,31
3,Michael Porter Jr.,"JAN 23, 2025",SAC,Vitória,Casa,20,8,0,123 - 132,8,2,34
4,Michael Porter Jr.,"JAN 21, 2025",PHI,Vitória,Casa,19,6,1,109 - 144,6,3,25
5,Michael Porter Jr.,"JAN 19, 2025",ORL,Vitória,Fora,15,6,0,100 - 113,5,3,28
6,Michael Porter Jr.,"JAN 17, 2025",MIA,Vitória,Fora,14,9,2,113 - 133,7,4,29
7,Michael Porter Jr.,"JAN 15, 2025",HOU,Derrota,Casa,8,3,0,128 - 108,2,0,24
8,Michael Porter Jr.,"JAN 14, 2025",DAL,Vitória,Fora,13,6,2,99 - 118,4,3,26
9,Michael Porter Jr.,"JAN 12, 2025",DAL,Vitória,Fora,13,4,1,101 - 112,8,1,25


In [26]:
adicionar_placares_no_dataset(estatisticas_jogador_3)
estatisticas_jogador_3 = estatisticas_jogador_3.drop(columns=['Id do jogo'])

estatisticas_jogador_3

,Jogador,Data do jogo,Adversário,Vitória ou derrota,Casa ou fora,Pontos,Rebotes,Assistências,Placar,Tentativas de cesta de 3 pontos,Cestas de 3 pontos marcadas,Tempo de permanência
0,Jamal Murray,"JAN 29, 2025",NYK,Derrota,Fora,33,5,6,122 - 112,7,3,37
1,Jamal Murray,"JAN 27, 2025",CHI,Derrota,Fora,18,6,6,129 - 121,5,1,40
2,Jamal Murray,"JAN 25, 2025",MIN,Derrota,Fora,25,2,5,133 - 104,5,3,35
3,Jamal Murray,"JAN 23, 2025",SAC,Vitória,Casa,18,6,5,123 - 132,6,1,40
4,Jamal Murray,"JAN 21, 2025",PHI,Vitória,Casa,9,6,7,109 - 144,3,0,33
5,Jamal Murray,"JAN 19, 2025",ORL,Vitória,Fora,19,2,4,100 - 113,6,1,38
6,Jamal Murray,"JAN 17, 2025",MIA,Vitória,Fora,30,7,8,113 - 133,7,2,37
7,Jamal Murray,"JAN 15, 2025",HOU,Derrota,Casa,22,4,5,128 - 108,5,4,30
8,Jamal Murray,"JAN 14, 2025",DAL,Vitória,Fora,45,2,6,99 - 118,9,5,37
9,Jamal Murray,"JAN 12, 2025",DAL,Vitória,Fora,17,2,2,101 - 112,6,2,32


In [31]:
estatisticas_jogadores = pd.concat([estatisticas_jogador_1, estatisticas_jogador_2, estatisticas_jogador_3], ignore_index=True)

estatisticas_jogadores['Data do jogo'] = pd.to_datetime(estatisticas_jogadores['Data do jogo'], format='%b %d, %Y')
estatisticas_jogadores = estatisticas_jogadores.sort_values(by='Data do jogo')

estatisticas_jogadores

,Jogador,Data do jogo,Adversário,Vitória ou derrota,Casa ou fora,Pontos,Rebotes,Assistências,Placar,Tentativas de cesta de 3 pontos,Cestas de 3 pontos marcadas,Tempo de permanência
128,Jamal Murray,2024-10-24,OKC,Derrota,Casa,12,6,4,102 - 87,5,2,38
40,Nikola Jokić,2024-10-24,OKC,Derrota,Casa,16,12,13,102 - 87,3,1,35
87,Michael Porter Jr.,2024-10-24,OKC,Derrota,Casa,15,8,2,102 - 87,10,3,32
39,Nikola Jokić,2024-10-26,LAC,Derrota,Casa,41,9,4,109 - 104,12,7,37
86,Michael Porter Jr.,2024-10-26,LAC,Derrota,Casa,9,9,3,109 - 104,6,0,38
...,...,...,...,...,...,...,...,...,...,...,...,...
89,Jamal Murray,2025-01-27,CHI,Derrota,Fora,18,6,6,129 - 121,5,1,40
1,Nikola Jokić,2025-01-27,CHI,Derrota,Fora,33,12,14,129 - 121,4,1,40
41,Michael Porter Jr.,2025-01-29,NYK,Derrota,Fora,9,3,2,122 - 112,5,1,26
88,Jamal Murray,2025-01-29,NYK,Derrota,Fora,33,5,6,122 - 112,7,3,37


In [32]:
salvar_dataset_csv(estatisticas_jogadores, f"{RESULTADOS_PATH}/rf2_estatisticas_jogadores.csv")